In [4]:
from bs4 import BeautifulSoup,SoupStrainer
import requests
import pandas as pd
import re
import math
from pprint import pprint

import selenium
from selenium.webdriver.support.wait import WebDriverWait as wait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoAlertPresentException
from selenium.common.exceptions import TimeoutException
from selenium import webdriver
from selenium.webdriver.chrome.options import Options


In [ ]:

url = "https://www.automobiledimension.com/"
req = requests.get(url)
soup = BeautifulSoup(req.content, 'html.parser')
print(soup.prettify())


In [ ]:
all_cars=[]
for link in soup.find_all('a'):
    if re.search('.*-dimensions.html',str(link.get('href'))):
        car_brand=link.get('href')
        print(car_brand)
        url = "https://www.automobiledimension.com/{}".format(car_brand)
        req = requests.get(url)
        sub_soup = BeautifulSoup(req.content, 'html.parser')        
        for to_parse in sub_soup.body.main.find_all("div",{"class":"unit"}):
            dict1={}
            dict1['brand']=car_brand.replace('-car-dimensions.html','')
            dict1['car make']=to_parse.h2.text
            dict1.update(dict(zip(['L','W','H'],re.split('x',re.search('L x W x H: (\d{3,5} x \d{3,5} x \d{3,5}) mm',str(to_parse)).group(1)))))
            try:
                dict1['boot space']=re.search('Boot capacity: (\d{0,4}).* dm',str(to_parse)).group(1)
            except:
                print("No boot space info for {}".format(to_parse.h2.text))
                
            # Go to car's specific page
            car_page=[x.get('href') for x in to_parse.find_all("a")][0]
            url = "https://www.automobiledimension.com/{}".format(car_page)
            dict1['link']=url
            req = requests.get(url)
            page_soup = BeautifulSoup(req.content, 'html.parser')        
            
            # Return the free text
            car_text=page_soup.find("div", {"class": "interior-text"})
            if car_text:
                #Figure out what type of car it is
                car_classes=[x.contents for x in car_text.find_all("a")]
                try:
                    if car_classes[-1][0]=='previous model':
                        dict1['type']=car_classes[-2][0]
                    else:
                        dict1['type']=car_classes[-1][0]
                except:
                    pass
                
                # Figure out petrol type
                dict1['petrol types']=','.join([x["alt"] for x in car_text.find_all("img")])
                
            # Return the caption under the pic of seats saying how many seats
            for x in page_soup.find_all("figure", {"class": "interior-figure"}):
                if re.search('Interior space with \d seats',str(x)):
                    dict1['num seats']=re.search('Interior space with (\d) seats',str(x)).group(1)          
            all_cars.append(dict1)

In [ ]:
pd.options.display.max_rows=999
pd.set_option('display.max_colwidth', -1)

cars_df=pd.DataFrame(all_cars)

cars_df=cars_df[cars_df['boot space'].notnull()]
cars_df=cars_df[cars_df['boot space'].str.isnumeric()]
cars_df['boot space']=pd.to_numeric(cars_df['boot space'])

cars_df=cars_df.sort_values(by='boot space',ascending=False)
cars_df

In [ ]:
writer = pd.ExcelWriter(r'C:\Users\steve\Desktop\Cars.xlsx', engine='xlsxwriter')
cars_df.to_excel(writer, index=False)
worksheet = writer.sheets['Sheet1']
